## Preambule

In [8]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import json

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Paths

In [9]:
path_data = Path("K:/Data/Data_effortsharing/DataUpdate_ongoing/startyear_2021/")
path_data_gen = Path("K:/Data/Data_effortsharing/DataUpdate_ongoing/")
path_figs = Path("K:/Code/effort-sharing/") / "Figures" / "Paper_Robiou2023"

## Read data files

In [10]:
xr_dataread = xr.open_dataset(path_data / "xr_dataread.nc")
xr_combs = xr.open_dataset(path_data / "xr_comb.nc")
xr_temps = xr.open_dataset(path_data / "xr_combtemps.nc")
all_regions_iso = np.load(path_data_gen / "all_regions.npy")
all_regions_names = np.array(list(np.load(path_data_gen / "all_regions_names.npy")[:-1])+['Earth']) # Only in old xr_dataread version, should be updated when reran
all_countries_iso = np.load(path_data_gen / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data_gen / "all_countries_names.npy", allow_pickle=True)
only_regions_iso = ['CVF', 'G20', "EU", "G7", "NA", "AU", "AF", "SIDS", "LDC", "WORLD"]
only_regions_names = ['Climate Vulnerability Forum','G20',"European Union","G7","Northern America","Australasia","African Group","Small Island Developing States","Least Developed Countries", 'World']

In [11]:
#xr_budgets_scenario = xr.open_dataset(path_data / "xr_budgets_scenario.nc")
#df_michel = pd.read_excel(Path("X:/user/dekkerm/Data/") / "NDC" / "NDC_comparison.xlsx", sheet_name="Michel")

In [12]:
df_g = pd.read_excel(Path("X:/user/dekkerm/Data/") / "UNFCCC_Parties_Groups_noeu.xlsx", sheet_name = "Country groups")
countries_iso = np.array(df_g["Country ISO Code"])
countries_name = np.array(df_g["Name"])
countries_iso = np.array(df_g["Country ISO Code"])
group_cvf = countries_iso[np.array(df_g["CVF (24/10/22)"]) == 1]
group_g20 = countries_iso[np.array(df_g["G20"]) == 1]
group_eu = countries_iso[np.array(df_g["EU"]) == 1]
group_g7 = countries_iso[np.array(df_g["G7"]) == 1]
group_na = countries_iso[np.array(df_g["Northern America"]) == 1]
group_um = countries_iso[np.array(df_g["Umbrella"]) == 1]
group_au = countries_iso[np.array(df_g["Australasia"]) == 1]
group_af = countries_iso[np.array(df_g["African Group"]) == 1]
group_sids = countries_iso[np.array(df_g["SIDS"]) == 1]
group_ldc = countries_iso[np.array(df_g["LDC"]) == 1]
group_eg = countries_iso[np.array(df_g["European Group"]) == 1]
group_world = np.copy(countries_iso)

## Compute 50% reduction years

In [13]:
reds_1 = xr_combs.Approach1_gdp.sel(Category='C1+C2', Scenario='SSP2', Discount_factor=0, Historical_startyear=1990) / xr_dataread.sel(Time=2020).GHG_hist_excl
reds_2 = xr_combs.Approach2.sel(Category='C1+C2', Scenario='SSP2', Discount_factor=0, Historical_startyear=1990) / xr_dataread.sel(Time=2020).GHG_hist_excl

list_cty = []
list_year1 = []
list_year2 = []
a = 0
for cty in all_countries_iso:
    list_cty.append(cty)
    if cty in group_eu:
        cty2 = 'EU'
    else:
        cty2 = cty
    year1 = reds_1.Time[np.where(reds_1.sel(Region=cty2) < 0.5)[0]]
    year2 = reds_2.Time[np.where(reds_2.sel(Region=cty2) < 0.5)[0]]
    if len(year1) > 0:
        list_year1.append(np.array(year1)[0])
    else:
        list_year1.append(2100)
    if len(year2) > 0:
        list_year2.append(np.array(year2)[0])
    else:
        list_year2.append(2100)
    
list_year1 = np.array(list_year1)
list_year2 = np.array(list_year2)
list_cty = np.array(list_cty)

## Plot

In [15]:
fig = make_subplots(rows=2, cols=1,
                    specs=[[{'type': 'choropleth'}],
                           [{'type': 'choropleth'}]],
                    horizontal_spacing = 0., vertical_spacing=0.0)

for i in range(2):
    dath = [list_year1, list_year2][i]
    dat = np.array(dath)
    scale = 'Spectral'
    dc = dict(orientation='h', x=0.35, y=0.52, xanchor='left', yanchor='bottom', title='<b>Year of 50% reduction', tickvals=np.arange(2020, 2101, 10),
                        tickfont=dict(size=14),
                        len=0.4, lenmode='fraction', thickness=20, thicknessmode='pixels', titlefont=dict(size=14))
    #Grey out nans
    fig.add_trace(go.Choropleth(
            locations=['ATA'],
            z = [0.5],
            locationmode = 'ISO-3',
            colorscale ='Greys',
            zmax = 4.5,
            zmin = 0.5,
            text = ['hoi'],
            hovertemplate  = '%{text}',
            name="",
            marker_line_color='black', 
            marker_line_width=0,
            showscale=False,
        ), [1, 2][i], 1)
    
    
    fig.add_trace(go.Choropleth(
            locations=list_cty,
            z = dat,
            locationmode = 'ISO-3',
            colorscale = scale,
            zmax = 2100,
            zmin = 2020,
            text = [str(r)+": "+str(np.round(dat[r_i], 1)) for r_i, r in enumerate(list_cty)],
            hovertemplate  = '%{text}',
            name="",
            colorbar=dc,
            marker_line_color='black', 
            marker_line_width=1,
            showscale=[True, False][i],
        ), [1, 2][i], 1)

fs=20
fig['layout'].update(
    annotations=[dict(x=0.15, y=1.02, text='<b>(a) Approach 1 (GDP)</b>', xref='paper', yref='paper', showarrow=False, font=dict(color='black', size=fs)),
                 dict(x=0.15, y=0.52, text='<b>(b) Approach 2', xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=fs)),
                 ])

fig.update_geos(visible=True,
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                showframe=False,
                resolution=110,
                landcolor="lightgrey",
                showcoastlines=False,)
fig.update_traces(colorbar=dict(
    title=dict(
        side="bottom"
    )
))

fig.update_layout(height=1500, width=1450)
fig.write_image(path_figs / "Figure_3.svg", scale=5)
fig.show()